In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random as rd

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [2]:
CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [3]:
def tsp_cost(tsp):
    #assert tsp[0] == tsp[-1]
    #assert set(tsp) == set(range(len(CITIES))) the controls are done in the loop cycle

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    
    tot_cost += DIST_MATRIX[tsp[-1], tsp[0]]
    return tot_cost

In [4]:
POPULATION_SIZE = 100

### get a starting greedy path

In [5]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True
tsp = list()
tsp.append(int(city))
while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    visited[closest] = True
    city = closest
    tsp.append(int(city))


### start from a randomized greedy population of paths


In [6]:
def partial_greedy_randomization(tsp, num_randomized):
    tsp = tsp.copy()  # Create a copy to avoid modifying the original
        
    indices = rd.sample(range(len(tsp)), num_randomized)
    randomized_part = [tsp[i] for i in indices]
    
    # Shuffle the selected segment
    rd.shuffle(randomized_part)
    
    # Insert the shuffled segment back into the original list
    for i, idx in enumerate(indices):
        tsp[idx] = randomized_part[i]
    
    return tsp

In [7]:
population = []
while len(population) < POPULATION_SIZE:
    rd_tsp = partial_greedy_randomization(tsp,10)
    rd_tsp.append(rd_tsp[0]) # always return to the starting city
    if set(rd_tsp[:-1]) == set(range(len(CITIES))):
        population.append(rd_tsp)
    

### define mutation function for the offspring

In [8]:
def insert_mutation(tsp):
    i,j = rd.randint(1,len(tsp)-1), rd.randint(1,len(tsp)-1)
    # insert mutation
    if i != j:
        if i < j:
            city_to_move = tsp[j]
            tsp.pop(j)
            tsp.insert(i+1,city_to_move)
        elif i > j:
            city_to_move = tsp[i]
            tsp.pop(i)
            tsp.insert(j+1,city_to_move)
    return tsp 

### define inver over crossover to generate the offspring

In [9]:
def io_crossover(p1, p2):
    # indexes extraction
    i = np.random.randint(1, len(p1)-1)
    j = np.random.randint(i, len(p1)-1)

    
    o = [v for v in p1] # offspring

    if np.random.rand() < 0.4: # I use only p1 with probability 0.4 (indexes are i and j), otherwise I use also p2
        segment = p1[i:j]
        segment = segment[::-1]
        o[i:j] = segment
    else:
        node = p1[i] # value to search in p2
        el_to_find = p2[p2.index(node)+1] # element next to the previous value in p2
        node2 = p1.index(el_to_find) # index of the previously found element in p1
        if node2 > i: 
            segment = p1[i:node2] # segment between the two nodes
            segment = segment[::-1]
            o[i:node2] = segment 
            o.pop(node2) # remove the second node 
            o.insert(i+1,el_to_find) # insert it next to the first and slide the  following elements
        elif i > node2:
            segment = p1[node2:i]
            segment = segment[::-1]
            o[node2:i] = segment
            o.pop(i)
            o.insert(node2+1,el_to_find)

    return o



### define fitness proportional selection function to select the parents

In [10]:
def fitness_proportional_parents(population, fitness): # k is the number of individuals to select
    total_fitness = sum(fitness)
    probabilities = [f / total_fitness for f in fitness]
    return rd.choices(population, probabilities, k=50)

def fitness_proportional_population(population, fitness): # k is the number of individuals to select
    total_fitness = sum(fitness)
    probabilities = [f / total_fitness for f in fitness]
    return rd.choices(population, probabilities, k=100)

### define the generational loop

In [11]:
for steps in range (100):
    costs = [tsp_cost(i) for i in population]
    parents = fitness_proportional_parents(population, costs) # parent selection (50 parents)

    for i in range(0, len(parents)-1,2): # take pairs of parents to do crossover

        p1,p2 = parents[i], parents[i+1]

        o1 = io_crossover(p1,p2)
        if np.random.random() < 0.3: # mutate the offspring with probability 0.3
            o1 = insert_mutation(o1)
           
        population.extend([o1])

    
    print("Generated Population:")
    for i, path in enumerate(population):
        print(f"Path {i}: {path}")

    # Verifica della correttezza dei percorsi prima di calcolare i costi
    for i,path in enumerate(population):
        if path[0] != path[-1]: 
            print(f"Path {i} does not return to the starting city.") #
        elif len(set(path[:-1])) != len(CITIES): 
            print(f"Path {i} does not cover all cities.")
            population.remove(path) # remove the path from the population
    costs = [tsp_cost(i) for i in population]
    population = fitness_proportional_population(population, costs) # prune the population to the desired size




Generated Population:
Path 0: [0, 33, 12, 41, 9, 4, 19, 32, 25, 28, 18, 20, 3, 6, 29, 45, 44, 30, 24, 5, 40, 43, 42, 13, 16, 23, 10, 26, 39, 34, 22, 14, 21, 35, 11, 1, 2, 38, 17, 31, 8, 37, 15, 7, 36, 27, 0]
Path 1: [0, 33, 12, 30, 38, 4, 43, 32, 25, 28, 18, 20, 9, 6, 44, 45, 23, 16, 41, 29, 19, 22, 42, 13, 3, 40, 10, 26, 39, 34, 15, 14, 21, 35, 11, 1, 2, 5, 17, 31, 8, 37, 24, 7, 36, 27, 0]
Path 2: [0, 33, 12, 30, 9, 13, 16, 32, 25, 28, 18, 20, 3, 6, 44, 45, 23, 43, 41, 5, 40, 22, 34, 14, 19, 29, 10, 26, 39, 8, 15, 42, 21, 35, 11, 1, 2, 38, 17, 31, 4, 37, 24, 7, 36, 27, 0]
Path 3: [33, 7, 12, 30, 9, 4, 19, 32, 25, 28, 18, 20, 3, 6, 34, 45, 23, 43, 41, 5, 40, 37, 42, 13, 16, 10, 44, 26, 39, 0, 15, 14, 21, 35, 11, 1, 2, 38, 17, 31, 8, 29, 24, 22, 36, 27, 33]
Path 4: [0, 15, 12, 30, 9, 4, 19, 32, 33, 28, 22, 21, 3, 6, 44, 45, 23, 43, 34, 5, 40, 41, 42, 13, 16, 29, 10, 26, 39, 20, 35, 14, 25, 18, 11, 1, 2, 38, 17, 31, 8, 37, 24, 7, 36, 27, 0]
Path 5: [0, 33, 12, 16, 9, 4, 7, 32, 25, 28, 18

ValueError: 23 is not in list